# Confidence Intervals

## Confidence Intervals Using NHANES Data
--- 
### Formulas:
##### Confidence Intervals
**Population Proportions**

$\hat{p} \pm z^**\sqrt{\frac{\hat{p}(1-\hat{p})}{n}} $

**Population Mean**

$\bar{x} \pm t^* * (\frac{\sigma}{\sqrt{n}}) $





In [2]:
# Initial Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.api as sm
%matplotlib inline